In [10]:
import os
import numpy as np
import scipy.sparse.csgraph as csg
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import learning_util as lu

In [11]:
# Distortion calculations

def acosh(x):
    return torch.log(x + torch.sqrt(x**2-1))

def dist_h(u,v):
    z  = 2*torch.norm(u-v,2)**2
    uu = 1. + torch.div(z,((1-torch.norm(u,2)**2)*(1-torch.norm(v,2)**2)))
    return acosh(uu)

def distance_matrix_euclidean(input):
    row_n = input.shape[0]
    mp1 = torch.stack([input]*row_n)
    mp2 = torch.stack([input]*row_n).transpose(0,1)
    dist_mat = torch.sum((mp1-mp2)**2,2).squeeze()
    return dist_mat

def distance_matrix_hyperbolic(input):
    row_n = input.shape[0]
    dist_mat = torch.zeros(row_n, row_n, device=device)
    for row in range(row_n):
        for i in range(row_n):
            if i != row:
                dist_mat[row, i] = dist_h(input[row,:], input[i,:])
    return dist_mat

def entry_is_good(h, h_rec): return (not torch.isnan(h_rec)) and (not torch.isinf(h_rec)) and h_rec != 0 and h != 0

def distortion_entry(h,h_rec):
    avg = abs(h_rec - h)/h
    return avg

def distortion_row(H1, H2, n, row):
    avg, good = 0, 0
    for i in range(n):
        if i != row and entry_is_good(H1[i], H2[i]):
            _avg = distortion_entry(H1[i], H2[i])
            good        += 1
            avg         += _avg
    if good > 0:
        avg /= good 
    else:
        avg, good = torch.tensor(0., device=device, requires_grad=True), torch.tensor(0., device=device, requires_grad=True)
    return (avg, good)

def distortion(H1, H2, n, jobs=16):
#     dists = Parallel(n_jobs=jobs)(delayed(distortion_row)(H1[i,:],H2[i,:],n,i) for i in range(n))
    dists = (distortion_row(H1[i,:],H2[i,:],n,i) for i in range(n))
    to_stack = [tup[0] for tup in dists]
    avg = torch.stack(to_stack).sum()/n
    return avg


#Loading the graph and getting the distance matrix.

def load_graph(file_name, directed=False):
    G = nx.DiGraph() if directed else nx.Graph()
    with open(file_name, "r") as f:
        for line in f:
            tokens = line.split()
            u = int(tokens[0])
            v = int(tokens[1])
            if len(tokens) > 2:
                w = float(tokens[2])
                G.add_edge(u, v, weight=w)
            else:
                G.add_edge(u,v)
    return G


def compute_row(i, adj_mat): 
    return csg.dijkstra(adj_mat, indices=[i], unweighted=True, directed=False)

def get_dist_mat(G):
    n = G.order()
    adj_mat = nx.to_scipy_sparse_matrix(G, nodelist=list(range(G.order())))
    t = time.time()
    
    num_cores = multiprocessing.cpu_count()
    dist_mat = Parallel(n_jobs=num_cores)(delayed(compute_row)(i,adj_mat) for i in range(n))
    dist_mat = np.vstack(dist_mat)
    return dist_mat


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [12]:
class Vocab:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0

    def addSentence(self, sentence):
        for token in sentence:
            self.addWord(token['form'])

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [13]:
from conllu import parse_tree, parse_tree_incr, parse, parse_incr
from io import open
import scipy.sparse.csgraph as csg
import networkx as nx
from collections import defaultdict
import json
import string


def unroll(node, G):
    if len(node.children) != 0:
        for child in node.children:
            G.add_edge(node.token['id'], child.token['id'])
            unroll(child, G)
    return G

sentences = []
data_file = open("UD_English-EWT/en_ewt-ud-train.conllu", "r", encoding="utf-8")
for sentence in parse_incr(data_file):
    sentences.append(sentence)
    
MIN_LENGTH = 10
MAX_LENGTH = 50

def check_length(sentence):
    return len(sentence) < MAX_LENGTH and len(sentence) > MIN_LENGTH 

def filterSentences(sentences):
    return [sent for sent in sentences if check_length(sent)]

input_vocab = Vocab("ewt_train_trimmed")
filtered_sentences = filterSentences(sentences)

sentences_text = []
for sent in filtered_sentences:
    input_vocab.addSentence(sent)
    sentences_text.append(sent.metadata['text'])
    
dev_dict  = {}
for idx in range(0, len(filtered_sentences)):
    curr_tree = filtered_sentences[idx].to_tree()
    G_curr = nx.Graph()
    G_curr = unroll(curr_tree, G_curr)
    G = nx.relabel_nodes(G_curr, lambda x: x-1)
    nx.write_edgelist(G, "train/"+str(idx)+".edges", data=False)
    G_final = nx.convert_node_labels_to_integers(G_curr, ordering = "decreasing degree")
    nx.write_edgelist(G_final, "ewt_train/"+str(idx)+".edges", data=False)
    dev_dict[idx] = list(G_final.edges)



In [14]:
def indexesFromSentence(vocab, sentence):
    return [vocab.word2index[token['form']] for token in sentence]

def tensorFromSentence(vocab, sentence):
    indexes = indexesFromSentence(vocab, sentence)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def pairfromidx(idx):
    input_tensor = tensorFromSentence(input_vocab, filtered_sentences[idx])
    G = load_graph("random_trees/"+str(idx)+".edges")
    target_matrix = get_dist_mat(G)
    target_tensor = torch.from_numpy(target_matrix).float().to(device)
    target_tensor.requires_grad = False
    n = G.order()
    return (input_tensor, target_tensor, n, sentences_text[idx])



In [15]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    

class Attention(nn.Module):
    def __init__(self, input_size, hidden_size, max_length=MAX_LENGTH):
        super(Attention, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.max_length = max_length
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)


    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        attention_scores = self.attn(torch.cat((embedded[0], hidden.unsqueeze(0)), 1))
        attn_weights = F.softmax(attention_scores, dim=0)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        return output

In [16]:
#Hyperbolic modules.

class HypLinear(nn.Module):
    """Applies a hyperbolic "linear" transformation to the incoming data: :math:`y = xA^T + b`
       Uses hyperbolic formulation of addition, scaling and matrix multiplication.
    """

    def __init__(self, in_features, out_features, bias=True):
        super(HypLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))

        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(1, out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input_):
        result = lu.torch_hyp_add(lu.torch_mv_mul_hyp(torch.transpose(self.weight,0,1), input_), self.bias) #(batch, input) x (input, output)
        return result

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )


In [17]:
#Trains a Euclidean LSTM.

def trainVanilla(input_tensor, ground_truth, n, encoder, encoder_optimizer, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
 
    input_length = input_tensor.size(0)
    target_length = ground_truth.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    final_embeddings = torch.zeros(input_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    
    dist_recovered = distance_matrix_euclidean(encoder_outputs)
    loss += distortion(ground_truth, dist_recovered, n)
    loss.backward()
    encoder_optimizer.step()

    return loss.item()

In [18]:
# Trains Euclidean LSTM + Attention.

def trainWAttention(input_tensor, ground_truth, n, encoder, encoder_optimizer, attention, attention_optimizer, iter, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    attention_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = ground_truth.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    encoder_hiddens = torch.zeros(input_length, encoder.hidden_size, device=device)
    final_embeddings = torch.zeros(input_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        encoder_hiddens[ei] = encoder_hidden[0, 0]
        
    for idx in range(input_length):
        output = attention(input_tensor[idx], encoder_hiddens[idx], encoder_outputs)
        final_embeddings[idx] = output[0]
        
    dist_recovered = distance_matrix_euclidean(final_embeddings)
    loss += distortion(ground_truth, dist_recovered, n)
    loss.backward()
    encoder_optimizer.step()
    attention_optimizer.step()

    return loss.item(), final_embeddings

In [19]:
def trainEuclidean(encoder, attention, n_epochs=10, n_iters=500, print_every=50, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  
    plot_loss_total = 0  

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    attention_optimizer = optim.SGD(attention.parameters(), lr=learning_rate)
    training_pairs = [pairfromidx(idx) for idx in range(n_iters)]

    euclidean_emb_dict = {}
    for i in range(n_epochs):
        for iter in range(1, n_iters+1):     
            training_pair = training_pairs[iter-1]
            input_tensor = training_pair[0]
            print("input tensor", input_tensor)
            target_matrix = training_pair[1]
            n = training_pair[2]
            loss, final_embeddings = trainWAttention(input_tensor, target_matrix, n, encoder, encoder_optimizer, attention, attention_optimizer, iter-1)
            torch.save(final_embeddings, "saved_tensors_tree/"+str(iter-1)+".pt")
            euclidean_emb_dict[iter-1] = final_embeddings
    #         loss = train(input_tensor, target_matrix, n, encoder, encoder_optimizer)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
    
    return euclidean_emb_dict

# hidden_size = 100
# encoder = EncoderLSTM(input_vocab.n_words, hidden_size).to(device)
# attention = Attention(input_vocab.n_words, hidden_size).to(device)
# euclidean_emb_dict = trainEuclidean(encoder, attention)


In [37]:
euclidean_embeddings = {}
saved_tensors = os.listdir("tree_emb_saved/")
indices = []
for file in saved_tensors:
    idx = int(file.split(".")[0])
    indices.append(idx)
    euclidean_embeddings[idx] = torch.load("tree_emb_saved/"+str(file), map_location=torch.device('cpu'))

In [22]:
#Riemannian SGD

from torch.optim.optimizer import Optimizer, required
spten_t = torch.sparse.FloatTensor


def poincare_grad(p, d_p):
    """
    Calculates Riemannian grad from Euclidean grad.
    Args:
        p (Tensor): Current point in the ball
        d_p (Tensor): Euclidean gradient at p
    """
    if d_p.is_sparse:
        p_sqnorm = torch.sum(
            p.data[d_p._indices()[0].squeeze()] ** 2, dim=1,
            keepdim=True
        ).expand_as(d_p._values())
        n_vals = d_p._values() * ((1 - p_sqnorm) ** 2) / 4
        d_p = spten_t(d_p._indices(), n_vals, d_p.size())
    else:
        p_sqnorm = torch.sum(p.data ** 2, dim=-1, keepdim=True)
        d_p = d_p * ((1 - p_sqnorm) ** 2 / 4).expand_as(d_p)

    return d_p


def euclidean_grad(p, d_p):
    return d_p


def retraction(p, d_p, lr):
    # Gradient clipping.
    if torch.all(d_p < 1000) and torch.all(d_p>-1000):
        p.data.add_(-lr, d_p)


class RiemannianSGD(Optimizer):
    r"""Riemannian stochastic gradient descent.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        rgrad (Function): Function to compute the Riemannian gradient from
            an Euclidean gradient
        retraction (Function): Function to update the parameters via a
            retraction of the Riemannian gradient
        lr (float): learning rate
    """

    def __init__(self, params, lr=required, rgrad=required, retraction=required):
        defaults = dict(lr=lr, rgrad=rgrad, retraction=retraction)
        super(RiemannianSGD, self).__init__(params, defaults)

    def step(self, lr=None):
        """Performs a single optimization step.
        Arguments:
            lr (float, optional): learning rate for the current update.
        """
        loss = None

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if lr is None:
                    lr = group['lr']
                d_p = group['rgrad'](p, d_p)
                group['retraction'](p, d_p, lr)

        return loss

In [27]:
# Does Euclidean to hyperbolic mapping using series of FC layers.
# We use ground truth distance matrix for the pair since the distortion for hyperbolic embs are really low.

def trainFCHyp(input_matrix, ground_truth, n, mapping, mapping_optimizer, max_length=MAX_LENGTH):
    mapping_optimizer.zero_grad()
 
    loss = 0
    output = mapping(input_matrix.float())
    dist_recovered = distance_matrix_hyperbolic(output) 
    loss += distortion(ground_truth, dist_recovered, n)
    loss.backward()
    mapping_optimizer.step()

    return loss.item()


In [38]:
def trainFCIters(mapping, n_epochs=5, n_iters=500, print_every=50, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  
    plot_loss_total = 0  

    mapping_optimizer = RiemannianSGD(mapping.parameters(), lr=learning_rate, rgrad=poincare_grad, retraction=retraction)
    training_pairs = [pairfromidx(idx) for idx in range(n_iters)]

    for i in range(n_epochs):
        print("Starting epoch "+str(i))
        iter=1
        for idx in indices:     
            input_matrix = euclidean_embeddings[idx]
            target_matrix = training_pairs[idx][1]
            n = training_pairs[idx][2]
            loss = trainFCHyp(input_matrix, target_matrix, n, mapping, mapping_optimizer)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

            iter+=1
            
input_size = 10
output_size = 10
mapping = nn.Sequential(
          nn.Linear(input_size, 50).to(device),
          nn.ReLU().to(device),
          nn.Linear(50, output_size).to(device),
          nn.ReLU().to(device))
          
# trainFCIters(mapping)

In [13]:
def trainHyperbolic(input_tensor, ground_truth, n, encoder, encoder_optimizer, mapping, mapping_optimizer, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    mapping_optimizer.zero_grad()
 
    input_length = input_tensor.size(0)
    target_length = ground_truth.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    final_embeddings = torch.zeros(input_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        
    final_embeddings = mapping(encoder_outputs)

    dist_recovered = distance_matrix_hyperbolic(final_embeddings) 
    loss += distortion(ground_truth, dist_recovered, n)
    loss.backward()
    encoder_optimizer.step()
    mapping_optimizer.step()

    return loss.item()


In [16]:
#Does end to end hyperbolic training.

def trainHypIters(encoder, fc, n_iters=7600, print_every=500, plot_every=100, learning_rate=0.1):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  
    plot_loss_total = 0  

    encoder_optimizer = RiemannianSGD(encoder.parameters(), lr=learning_rate, rgrad=poincare_grad, retraction=retraction)
    fc_optimizer = RiemannianSGD(fc.parameters(), lr=learning_rate, rgrad=poincare_grad, retraction=retraction)
    training_pairs = [pairfromidx(idx) for idx in range(n_iters)]

    for iter in range(1, n_iters + 1):     
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_matrix = training_pair[1]
        n = training_pair[2]
        loss = trainHyperbolic(input_tensor, target_matrix, n, encoder, encoder_optimizer, fc, fc_optimizer)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0



In [39]:
hidden_size = 100
encoder = EncoderLSTM(input_vocab.n_words, hidden_size).to(device)
input_size = 100
output_size = 100
fc = nn.Sequential(
          nn.Linear(input_size, 1000).to(device),
          nn.ReLU().to(device),
          nn.Linear(1000, 500).to(device),
          nn.ReLU().to(device),
          nn.Linear(500, 50).to(device),
          nn.ReLU().to(device),
          nn.Linear(50, output_size).to(device))
          
# trainHypIters(encoder, fc)